# **Real-time Object Motion Detection and Tracking using OpenCV**
## **On a web cam live video**

In [ ]:
## Motion detection and tracking on a web cam live video.

import cv2
import numpy as np

# ---- Open web cam ---
vid = cv2.VideoCapture(0)

# ---- reading first two frames for initial frame differencing.-----
s,f1 = vid.read()
s,f2 = vid.read()

# ---- Creting named window and trackbars for thresholding. ----
cv2.namedWindow('Motion Object Detection',cv2.WINDOW_FREERATIO)

def nothing(x):
    pass
cv2.createTrackbar('Thresh_min','Motion Object Detection',0,255, nothing)


# ----- Main Loop for continuous iterations -----
while True:
    thr_min = cv2.getTrackbarPos('Thresh_min','Motion Object Detection')

    if not s:
        break

    # ----- frame differencing ----
    fr_diff  = cv2.absdiff(f1,f2)
    gray = cv2.cvtColor(fr_diff,cv2.COLOR_BGR2GRAY)  ### color conversion far faster processing.
    blur = cv2.GaussianBlur(gray, (5,5), 1) ### bluring to reduce noise.

    # ---- thresholding to find the edges -----
    thr_val, thr_fr = cv2.threshold(blur, thr_min, 255, cv2.THRESH_BINARY)
    
    # ----finding the contours----
    c, h = cv2.findContours(thr_fr, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

    # ----- draw all the contours with green color -----
    cv2.drawContours(f1, c, -1, (0, 255, 0), 2)

    # ---- drawing bounding box for moving objects ----
    if len(c) > 0:
        for i in c: ## looping through all contours
            if cv2.contourArea(i) > 100:   # ----- contours area greter than 100 wil be drawn.
                rect = cv2.minAreaRect(i)    ## ----- rectangle that having minimum area but covers all the area of contour
                points = cv2.boxPoints(rect).astype(int)    ## ---- gives points of x,y,width and height
                cv2.drawContours(f1, [points], -1, (0, 0, 255), 3)    ## ----- to draw contours
        
    # ----- showing the frames -----
    cv2.imshow('Motion Object Detection',f1)

    # ---- updating the frames -----
    f1 = f2
    s,f2 = vid.read()

    # ---- condition to exiting loop ----
    if cv2.waitKey(1) == ord('q'):
        break


# ---- clearing all windows -----
cv2.destroyAllWindows()